In [1]:
import numpy as np
import pylas
from collections import Counter
from scipy.spatial import ConvexHull, Delaunay
from shapely.geometry import LineString, Polygon, MultiPolygon,mapping
import fiona

# Open and prepare the data

In [ ]:
las_test = pylas.open('All_area_full.las')

In [2]:
# Read in LAS file and extract tree coordinates
# tree_coords should be a list of (x,y,z) tuples representing the tree positions
data1 = las_test.read()
array_test = data1.point_source_id
print("This las file has",len(data1.points),"points")
counter_object3 = Counter(array_test)
keys3 = counter_object3.keys()
num_values3 = len(keys3)
print("This las file has",num_values3,"individual trees")
tree_coord = {}
for i in keys3:
    tree=[]
    tree = np.c_[data1.x[data1.point_source_id == i],data1.y[data1.point_source_id == i]]
    tree_coord[i]=[]
    tree_coord[i].append(tree)
    tree=[]

This las file has 896981 points
This las file has 2375 individual trees


# Create convex hulls

In [3]:
hulls = []
for tree_id, coords in tree_coord.items():
    if len(coords[0]) > 3:
        hull = ConvexHull(coords[0])
        hulls.append(hull)

# Save the 2D border of individual trees as shapefiles

In [4]:

# Convert the ConvexHull objects to Polygon objects
hulls_polygons = [Polygon(hull.points[hull.vertices]) for hull in hulls]

# Set up the Shapefile schema
schema = {
    'geometry': 'LineString',
    'properties': {'id': 'int', 'hull_id': 'int'},
}
i=0
# Save the hull borders to a Shapefile
with fiona.open('Shapefile_output.shp', 'w', 'ESRI Shapefile', schema) as c:
    for g in hulls_polygons:
        i=i+1
        hull_border = LineString(g.exterior)
        c.write({
            'geometry': mapping(hull_border),
            'properties': {'id': 1, 'hull_id': i},
        })
